In [ ]:
!pip install autopep8
!pip install jupyter_contrib_nbextensions==0.5.1
!pip install bokeh==0.12.15

In [38]:
import json
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler

from dwave.system.composites import EmbeddingComposite

# Please enter the S3 bucket you created during onboarding in the code below
my_bucket = "amazon-braket-zolnierczyk" # the name of the bucket
my_prefix = "factoring" # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)

# session and device
device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6")
print('Device:', device)

Device: Device('name': DW_2000Q_6, 'arn': arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6)


In [39]:
import dwavebinarycsp as dbc

and_gate = dbc.factories.and_gate(["x1", "x2", "x3"])
and_csp = dbc.ConstraintSatisfactionProblem('BINARY')
and_csp.add_constraint(and_gate)
# Test the CSP
and_csp.check({"x1": 1, "x2": 1, "x3": 1})

True

In [40]:
import itertools

configurations = []
for (x1, x2, x3) in  list(itertools.product([0, 1], repeat=3)):
     E = 3*x3+x1*x2-2*x1*x3-2*x2*x3
     configurations.append((E, x1, x2, x3))

configurations.sort()

print("E, x1, x2, x3")
configurations

E, x1, x2, x3


[(0, 0, 0, 0),
 (0, 0, 1, 0),
 (0, 1, 0, 0),
 (0, 1, 1, 1),
 (1, 0, 1, 1),
 (1, 1, 0, 1),
 (1, 1, 1, 0),
 (3, 0, 0, 1)]

In [41]:
and_bqm = dbc.stitch(and_csp)
and_bqm.remove_offset()

print("Linear coefficients:\n\t{}".format(
      {key: round(val, 2) for key, val in and_bqm.linear.items()}))
print("Quadratic coefficients:\n\t{}".format(
      {key: round(val, 2) for key, val in and_bqm.quadratic.items()}))

Linear coefficients:
	{'x1': -0.0, 'x2': -0.0, 'x3': 6.0}
Quadratic coefficients:
	{('x1', 'x2'): 2.0, ('x1', 'x3'): -4.0, ('x2', 'x3'): -4.0}


In [42]:
from dimod import ExactSolver

sampler = ExactSolver()
solution = sampler.sample(and_bqm)

print(solution)

  x1 x2 x3 energy num_oc.
5  1  1  1   -0.0       1
1  1  0  0   -0.0       1
3  0  1  0   -0.0       1
0  0  0  0    0.0       1
6  1  0  1    2.0       1
4  0  1  1    2.0       1
2  1  1  0    2.0       1
7  0  0  1    6.0       1
['BINARY', 8 rows, 8 samples, 3 variables]


In [43]:
P = 21   
module_lenght = 6
bP = "{:06b}".format(P)    # "{:06b}" formats for 6-bit binary
print(bP)

csp = dbc.factories.multiplication_circuit(int(module_lenght/2))

print(next(iter(csp.constraints)))

bqm = dbc.stitch(csp, min_classical_gap=.1)

print("BQM has {} variables: \n\t{}".format(len(bqm.variables), list(bqm.variables)))

010101
Constraint.from_configurations(frozenset({(1, 0, 0), (1, 1, 1), (0, 1, 0), (0, 0, 0)}), ('a0', 'b0', 'p0'), Vartype.BINARY, name='AND(a0, b0) = p0')
BQM has 27 variables: 
	['a0', 'b0', 'p0', 'and0,1', 'b1', 'and0,2', 'b2', 'a1', 'and1,0', 'p1', 'carry1,0', 'and1,1', 'sum1,1', 'carry1,1', 'and1,2', 'a2', 'and2,0', 'p2', 'carry2,0', 'and2,1', 'sum2,1', 'carry2,1', 'and2,2', 'p3', 'carry3,0', 'p4', 'p5']


In [44]:
p_vars = ['p'+str(i) for i in range(module_lenght)]

# Convert P from decimal to binary
fixed_variables = dict(zip(reversed(p_vars), "{:06b}".format(P)))
fixed_variables = {var: int(x) for(var, x) in fixed_variables.items()}

# Fix product variables
for var, value in fixed_variables.items():
    bqm.fix_variable(var, value)
    
print("BQM has {} non-fixed variables: \n\t{}".format(len(bqm.variables), list(bqm.variables)))

BQM has 21 non-fixed variables: 
	['a0', 'b0', 'and0,1', 'b1', 'and0,2', 'b2', 'a1', 'and1,0', 'carry1,0', 'and1,1', 'sum1,1', 'carry1,1', 'and1,2', 'a2', 'and2,0', 'carry2,0', 'and2,1', 'sum2,1', 'carry2,1', 'and2,2', 'carry3,0']


In [45]:
sampler = BraketDWaveSampler(s3_folder,'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6')

In [50]:
embeddings = {
    "SIMULATED_2000Q_CLOUD_TEST": {
        "a0": [431, 424, 175, 302, 168, 296],
        "a1": [148, 172, 156, 164],
        "a2": [417, 292, 289],
        "and0,1": [173, 165],
        "and0,2": [155, 167, 159],
        "and1,0": [171, 174],
        "and1,1": [152],
        "and1,2": [407, 415, 145, 273, 401],
        "and2,0": [303, 287, 295],
        "and2,1": [276, 284],
        "and2,2": [422, 419, 420],
        "b0": [169, 300, 297],
        "b1": [280, 162, 290, 286, 294],
        "b2": [160, 288, 416, 423],
        "carry1,0": [293, 291, 166, 163],
        "carry1,1": [400, 272, 144, 154, 149, 157],
        "carry2,0": [283, 285],
        "carry2,1": [275, 403, 406, 414],
        "carry3,0": [411, 412],
        "sum1,1": [281, 158, 153],
        "sum2,1": [282, 410],
    },
    "SIMULATED_2000Q_CLOUD_ALPHA": {
        "a0": [431, 424, 175, 302, 168, 296],
        "a1": [148, 172, 156, 164],
        "a2": [417, 292, 289],
        "and0,1": [173, 165],
        "and0,2": [155, 167, 159],
        "and1,0": [171, 174],
        "and1,1": [152],
        "and1,2": [407, 415, 145, 273, 401],
        "and2,0": [303, 287, 295],
        "and2,1": [276, 284],
        "and2,2": [422, 419, 420],
        "b0": [169, 300, 297],
        "b1": [280, 162, 290, 286, 294],
        "b2": [160, 288, 416, 423],
        "carry1,0": [293, 291, 166, 163],
        "carry1,1": [400, 272, 144, 154, 149, 157],
        "carry2,0": [283, 285],
        "carry2,1": [275, 403, 406, 414],
        "carry3,0": [411, 412],
        "sum1,1": [281, 158, 153],
        "sum2,1": [282, 410],
    },
    "DW_2000Q_5": {
        "and2,1": [2033, 1777, 1905],
        "sum1,1": [1772, 1770],
        "carry1,0": [1513, 1514, 1642, 1524, 1516],
        "and2,2": [1918, 1914, 1916],
        "a1": [1650, 1778],
        "and2,0": [1640, 1768, 1646],
        "and0,1": [1521, 1520, 1527],
        "a0": [1776, 1648],
        "and0,2": [1775, 1783, 1791],
        "carry2,1": [1904, 1908],
        "carry2,0": [1641, 1897, 1769],
        "sum2,1": [1901, 1907, 1909],
        "a2": [1774, 1782, 1790, 1658, 1786],
        "b0": [1662, 1654],
        "b1": [1649, 1661, 1657, 1653],
        "b2": [1780, 1787, 1915, 1788],
        "and1,2": [2043, 2045, 2037, 2034, 1906],
        "carry3,0": [1902, 1910],
        "and1,0": [1655, 1523, 1651],
        "and1,1": [1781, 1773, 1785, 1789],
        "carry1,1": [1898, 1784, 1903, 1912, 1919, 1911],
    },
    "DW_2000Q_6": {
        "and2,1": [590, 582, 450, 578],
        "sum1,1": [469, 464, 592, 720],
        "carry1,0": [727, 851, 723],
        "and2,2": [577, 449, 455, 448],
        "a1": [707, 711, 719],
        "and2,0": [709, 717, 725],
        "and0,1": [854, 843, 846],
        "a0": [585, 841, 713],
        "and0,2": [456, 597, 584, 589],
        "carry2,1": [457, 454, 462],
        "carry2,0": [722, 594, 466],
        "sum2,1": [452, 467, 468, 460],
        "a2": [710, 704, 576],
        "b0": [840, 712, 718],
        "b1": [706, 834, 842, 839, 847],
        "b2": [579, 583, 591],
        "and1,2": [715, 587, 459],
        "carry3,0": [471, 463],
        "and1,0": [835, 837, 853, 845],
        "and1,1": [586, 724, 716, 714],
        "carry1,1": [458, 453, 461],
    },
    "BAY17_P16_ALPHA": {
        "a0": [566, 5042],
        "b0": [521],
        "b1": [506],
        "and0,1": [5057],
        "b2": [5041],
        "and0,2": [416, 4922],
        "a1": [5102, 431],
        "and1,0": [536],
        "a2": [445, 5117, 446],
        "and2,0": [4952],
        "and1,1": [5026],
        "and2,1": [4996],
        "carry1,0": [551, 4967],
        "and1,2": [5056, 371],
        "and2,2": [326, 5116],
        "sum1,1": [401],
        "carry1,1": [341, 5011],
        "carry2,0": [4891, 430],
        "sum2,1": [4981],
        "carry2,1": [356],
        "carry3,0": [386, 4966],
    },
    "BAY17_P16_TEST": {
        "a0": [566, 5042],
        "b0": [521],
        "b1": [506],
        "and0,1": [5057],
        "b2": [5041],
        "and0,2": [416, 4922],
        "a1": [5102, 431],
        "and1,0": [536],
        "a2": [445, 5117, 446],
        "and2,0": [4952],
        "and1,1": [5026],
        "and2,1": [4996],
        "carry1,0": [551, 4967],
        "and1,2": [5056, 371],
        "and2,2": [326, 5116],
        "sum1,1": [401],
        "carry1,1": [341, 5011],
        "carry2,0": [4891, 430],
        "sum2,1": [4981],
        "carry2,1": [356],
        "carry3,0": [386, 4966],
    },
}
from dwave.system import FixedEmbeddingComposite


sampler_fixed_embedding = FixedEmbeddingComposite(sampler, embeddings['DW_2000Q_6'])
print("Mapping from BQM variables to qubits for the Dwave system:\n\n\t{}", sampler_fixed_embedding.embedding)




Mapping from BQM variables to qubits for the Dwave system:

	{} {'and2,1': (590, 582, 450, 578), 'sum1,1': (469, 464, 592, 720), 'carry1,0': (727, 851, 723), 'and2,2': (577, 449, 455, 448), 'a1': (707, 711, 719), 'and2,0': (709, 717, 725), 'and0,1': (854, 843, 846), 'a0': (585, 841, 713), 'and0,2': (456, 597, 584, 589), 'carry2,1': (457, 454, 462), 'carry2,0': (722, 594, 466), 'sum2,1': (452, 467, 468, 460), 'a2': (710, 704, 576), 'b0': (840, 712, 718), 'b1': (706, 834, 842, 839, 847), 'b2': (579, 583, 591), 'and1,2': (715, 587, 459), 'carry3,0': (471, 463), 'and1,0': (835, 837, 853, 845), 'and1,1': (586, 724, 716, 714), 'carry1,1': (458, 453, 461)}


In [54]:
from dwave.system import EmbeddingComposite
sampleset = sampler_fixed_embedding.sample(bqm, num_reads=100)
#sampleset = EmbeddingComposite(sampler).sample(bqm, num_reads=8000)
print("Best solution found: \n",sampleset.first.sample)

def to_base_ten(sample): 
    a = b = 0

    # we know that multiplication_circuit() has created these variables
    a_vars = ["a"+str(i) for i in range(int(module_lenght/2))]
    b_vars = ["b"+str(i) for i in range(int(module_lenght/2))]

    for lbl in reversed(a_vars):
        a = (a << 1) | sample[lbl]
    for lbl in reversed(b_vars):
        b = (b << 1) | sample[lbl]

    return a, b

a, b = to_base_ten(sampleset.first.sample)

from collections import OrderedDict



print("Given integer P={}, found factors a={} and b={}".format(P, a, b))

Best solution found: 
 {'a0': 1, 'a1': 1, 'a2': 1, 'and0,1': 1, 'and0,2': 0, 'and1,0': 1, 'and1,1': 1, 'and1,2': 0, 'and2,0': 1, 'and2,1': 1, 'and2,2': 0, 'b0': 1, 'b1': 1, 'b2': 0, 'carry1,0': 1, 'carry1,1': 0, 'carry2,0': 1, 'carry2,1': 0, 'carry3,0': 1, 'sum1,1': 1, 'sum2,1': 1}
OrderedDict([((7, 3), -0.0), ((3, 7), 0.0), ((5, 5), 1.0), ((5, 1), 1.0), ((5, 7), 2.0), ((3, 3), 2.0), ((5, 3), 2.0), ((3, 5), 2.0), ((1, 3), 2.0), ((7, 1), 2.0), ((7, 7), 3.0), ((4, 3), 3.0), ((3, 4), 3.0), ((7, 5), 4.0), ((1, 7), 4.0), ((3, 1), 4.0), ((1, 5), 5.0)])
Given integer P=21, found factors a=7 and b=3
